In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')



import random
import math
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [17]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [18]:
jittering = transforms.ColorJitter(brightness = 0.4, contrast = 0.4, saturation = 0.4)


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [19]:

transform_train1 = transforms.Compose([transforms.ToTensor(),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                                  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                      transforms.RandomGrayscale(p=1)])






transform  = transforms.Compose([
        transforms.ToTensor(),
             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [20]:
trainset1 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=transform_train1)

trainset2 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=transform)


vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=False,
                                        transform=transform)



trainset = trainset1 +trainset2

In [21]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/images/Statistical_Deep_Image",
                                           transform = transform)

In [22]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 64,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

모델 넣기 

In [23]:
class BasicBlock(nn.Module):
   def __init__(self, in_channels, out_channels, kernel_size=3):
       super(BasicBlock, self).__init__()



       self.c1 = nn.Conv2d(in_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)
       self.c2 = nn.Conv2d(out_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)

       self.downsample = nn.Conv2d(in_channels, out_channels, 
                                   kernel_size=1)
       

       self.bn1 = nn.BatchNorm2d(num_features=out_channels)
       self.bn2 = nn.BatchNorm2d(num_features=out_channels)

       self.relu = nn.ReLU()
   def forward(self, x):
       x_ = x

       x = self.c1(x)
       x = self.bn1(x)
       x = self.relu(x)
       x = self.c2(x)
       x = self.bn2(x)


       x_ = self.downsample(x_)


       x += x_
       x = self.relu(x)

       return x


class ResNet(nn.Module):
   def __init__(self, num_classes=10):
       super(ResNet, self).__init__()


       self.b1 = BasicBlock(in_channels=3, out_channels=64)
       self.b2 = BasicBlock(in_channels=64, out_channels=128)
       self.b3 = BasicBlock(in_channels=128, out_channels=256)


       self.pool = nn.AvgPool2d(kernel_size=2, stride=2) 

       self.fc1 = nn.Linear(in_features=4096, out_features=2048)
       self.fc2 = nn.Linear(in_features=2048, out_features=512)
       self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

       self.relu = nn.ReLU()
   def forward(self, x):
       x = self.b1(x)
       x = self.pool(x)
       x = self.b2(x)
       x = self.pool(x)
       x = self.b3(x)
       x = self.pool(x)

       x = torch.flatten(x, start_dim=1)

       x = self.fc1(x)
       x = self.relu(x)
       x = self.fc2(x)
       x = self.relu(x)
       x = self.fc3(x)

       return x

In [24]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [26]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [27]:
for epoch in range(50):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
    if i %1500 ==1499:
      with torch.no_grad():
        val_loss = 0.0
        for k, data1 in enumerate(vaild_loader, 0):
          val_inputs, val_label = data1[0].to(device), data1[1].to(device)
          val_output = net(val_inputs)
          v_loss = criterion(val_output, val_label)
          val_loss += v_loss
          _, predicted = torch.max(val_output.data,1)
          total += val_label.size(0)
          correct += (predicted == val_label).sum().item()
      print("validation loss {}".format(val_loss))
      print("vaildset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0
      with torch.no_grad():
        for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs= net(images)
          _, predicted = torch.max(outputs.data,1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print("testset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0

Epoch: 1,Batch : 750, Loss:0.01611057377367979
Epoch: 1,Batch : 1500, Loss:0.0130853780308571
validation loss 141.38055419921875
vaildset Accuracy  : 84.86
testset Accuracy  : 54.15
Epoch: 1,Batch : 2250, Loss:0.012356583467555993
Epoch: 1,Batch : 3000, Loss:0.009904100524613569
validation loss 139.92495727539062
vaildset Accuracy  : 85.1
testset Accuracy  : 53.45
Epoch: 2,Batch : 750, Loss:0.007910461362995648
Epoch: 2,Batch : 1500, Loss:0.00837292145057063
validation loss 144.1121826171875
vaildset Accuracy  : 85.12
testset Accuracy  : 54.0
Epoch: 2,Batch : 2250, Loss:0.009375771340030951
Epoch: 2,Batch : 3000, Loss:0.006920369636329951
validation loss 148.62342834472656
vaildset Accuracy  : 85.32
testset Accuracy  : 54.8
Epoch: 3,Batch : 750, Loss:0.006774282088605333
Epoch: 3,Batch : 1500, Loss:0.005991956056035178
validation loss 151.4351348876953
vaildset Accuracy  : 85.34
testset Accuracy  : 54.4
Epoch: 3,Batch : 2250, Loss:0.006040144728802261
Epoch: 3,Batch : 3000, Loss:0.0061

KeyboardInterrupt: ignored

pretrained 넣기


In [32]:
PATH = '/content/drive/MyDrive/share/trained_model/cifar_resnetmine.pth'



In [33]:
torch.save(net.state_dict(), PATH)

In [28]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

In [29]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    c= (predicted == labels).squeeze()
    for i in range(4):
      label= labels[i]
      class_correct[label]+= c[i].item()
      class_total[label]+= 1

for i in range(10):
  print("Accuracy of {} : {} %".format(classes[i],100* class_correct[i]/class_total[i]))

Accuracy of plane : 81.25 %
Accuracy of automobile : 58.333333333333336 %
Accuracy of bird : 33.333333333333336 %
Accuracy of cat : 33.333333333333336 %
Accuracy of deer : 66.66666666666667 %
Accuracy of dog : 25.0 %
Accuracy of frog : 25.0 %
Accuracy of horse : 33.333333333333336 %
Accuracy of ship : 68.75 %
Accuracy of truck : 58.333333333333336 %


In [30]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("trainset Accuracy  : {}".format(100* correct/total))

trainset Accuracy  : 99.98


In [31]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 55.15
